In [11]:
dataset = 'Musical_Instruments'

reviews_filepath = './data/raw_data/reviews_{0}_5.json.gz'.format(dataset)
metadata_filepath = './data/metadata/meta_{0}.json.gz'.format(dataset)

In [12]:
%load modules/scripts/Load\ datasets.py